In [1]:
import alive_progress
import numpy as np
import numpy.testing
import Py6S

from rtm_wrapper.engines.sixs import PySixSEngine, make_sixs_wrapper
from rtm_wrapper.execution import ConcurrentExecutor, SerialExecutor
from rtm_wrapper.simulation import Inputs, SweepSimulation

In [2]:
wl_spacing = 0.0025
wl = np.arange(0.4, 2.5 + wl_spacing, wl_spacing)

In [3]:
%%time
sweep = SweepSimulation(
    [
        ("wavelength", wl),
    ],
    base=Inputs(
        water=1,
        ozone=1,
        aot=[(5, 1)],
        wavelength=wl,
    ),
)

engine = PySixSEngine(make_sixs_wrapper())
runner = ConcurrentExecutor(max_workers=None)
# runner = SerialExecutor()

with alive_progress.alive_bar(sweep.sweep_size, force_tty=True) as bar:
    runner.run(sweep, engine, lambda _: bar())

outputs_a = runner.collect_results()

|████████████████████████████████████████| 841/841 [100%] in 20.3s (40.42/s)    
CPU times: user 5.78 s, sys: 6.23 s, total: 12 s
Wall time: 20.5 s


In [4]:
%%time
sixs = make_sixs_wrapper()
sixs.atmos_profile = Py6S.AtmosProfile.UserWaterAndOzone(1, 1)
sixs.aero_profile = Py6S.AeroProfile.UserProfile(Py6S.AeroProfile.Continental)
sixs.aero_profile.add_layer(5, 1)
_wl, outputs_b = Py6S.SixSHelpers.Wavelengths.run_wavelengths(sixs, wl)

Running for many wavelengths - this may take a long time
CPU times: user 5.05 s, sys: 5.95 s, total: 11 s
Wall time: 21.4 s


In [5]:
np.testing.assert_allclose(
    outputs_a.variables["apparent_radiance"], [o.apparent_radiance for o in outputs_b]
)